In [5]:
import os
import pandas as pd
import sys
import numpy as np
import sys
sys.path.insert(0, '../..')
import itertools

from JKBio import TerraFunction as terra
from ccle_processing.src.CCLE_postp_function import *
from JKBio import Helper as h
from JKBio.epigenetics import ChIP_helper as chip

import dalmatian as dm
import pyBigWig

from bokeh.plotting import *
from IPython.display import IFrame
import igv
from biomart import BiomartServer
import io

from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture

output_notebook()
%load_ext autoreload
%autoreload 2

Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [7]:
gsheet

,id,cell line,replicate,protein,quality,paired_end,matching input name,processed,name,previous name,...,unique mapped reads(droso),scaling factor,Unnamed: 25,Total QC,folderNarrow,folderCompensated,folderQC,folderBroad,folder Bigwig,folder diffPeaks
0,mp100,U937,1,INPUT,NaN,n,NaN,Y,mp100-U937-INPUT-r1,NaN,...,NaN,NaN,NaN,https://storage.cloud.google.com/amlproject/Ch...,https://console.cloud.google.com/storage/brows...,NaN,https://console.cloud.google.com/storage/brows...,https://console.cloud.google.com/storage/brows...,https://console.cloud.google.com/storage/brows...,NaN
1,mp101,NOMO1,1,INPUT,NaN,n,NaN,Y,mp101-NOMO1-INPUT-r1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,mp102,UT7,1,INPUT,NaN,n,NaN,Y,mp102-UT7-INPUT-r1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mp106,MV411,1,MYB,x,n,mp99,Y,mp106-MV411-MYB-r1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mp109,M6,1,CEBPA,x,n,mp98,Y,mp109-M6-CEBPA-r1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,mp916,MV411,1,E2F3,NaN,y,mp845,NaN,mp916-MV411-E2F3-r1,20201023_E2F3_MP8382_S41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
374,mp917,MV411,1,SREBP1,NaN,y,mp845,NaN,mp917-MV411-SREBP1-r1,20201023_SREBP1_MP8382_S42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
375,mp918,MV411_GFI_SC,1,FLAG_GFI,NaN,y,mp845,NaN,mp918-MV411_GFI_SC-FLAG_GFI-r1,20200903_1_MP8215_S101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376,mp919,MV411_MEF2D_NT_SC_63,3,FLAF_MEF2D,NaN,y,mp845,NaN,mp919-MV411_MEF2D_NT_SC_63-FLAF_MEF2D-r3,20200903_2_MP8215_S102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## v6

In [142]:
project="chipseq_MEF2D_degraded"

In [ ]:
! gsutil ls gs://transfer-amlproject/201001_MP8262_fastq/

In [ ]:
mkdir ../data/$project/ && mkdir ../data/$project/qc/

In [ ]:
!gsutil -m cp gs://transfer-amlproject/201001_MP8262_fastq/multiqc_report.html ../data/$project/qc/
!gsutil -m cp -r gs://transfer-amlproject/201001_MP8262_fastq/Reports/ ../data/$project/qc/
!gsutil -m cp -r gs://transfer-amlproject/201001_MP8262_fastq/multiqc_data/ ../data/$project/qc/

In [ ]:
! gsutil -m cp gs://transfer-amlproject/201001_MP8262_fastq/*  gs://transfer-amlproject/$project/

### analysis

In [ ]:
! mkdir ../../data/$project && mkdir ../../data/$project/fastqs && gsutil -m cp gs://transfer-amlproject/$project/* ../../data/$project/fastqs

In [ ]:
! rm ../../data/$project/fastqs/multiqc_report.html

In [ ]:
a = ! ls ../../data/$project/fastqs
a

In [ ]:
rename = {
"20201001_FLAG_DMSO_MP8262_S93": "mp881-MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D-r3",
"20201001_FLAG_VHL_MP8262_S99": "mp882-MV411_MEF2D_NT_SC_63_VHL-FLAG_MEF2D-r1",
"20201001_IRF8_DMSO_MP8262_S88": "mp883-MV411_MEF2D_NT_SC_63_DMSO-IFR8-r1",
"20201001_IRF8_VHL_MP8262_S94": "mp884-MV411_MEF2D_NT_SC_63_VHL-IFR8-r1",
"20201001_MED1_DMSO_MP8262_S91": "mp885-MV411_MEF2D_NT_SC_63_DMSO-MED1-r1",
"20201001_MED1_VHL_MP8262_S97": "mp886-MV411_MEF2D_NT_SC_63_VHL-MED1-r1",
"20201001_MEF2C_DMSO_MP8262_S90": "mp887-MV411_MEF2D_NT_SC_63_DMSO-MEF2C-r1",
"20201001_MEF2C_VHL_MP8262_S96": "mp888-MV411_MEF2D_NT_SC_63_VHL-MEF2C-r1",
"20201001_MYC_DMSO_MP8262_S89": "mp889-MV411_MEF2D_NT_SC_63_DMSO-MYC-r1",
"20201001_MYC_VHL_MP8262_S95": "mp890-MV411_MEF2D_NT_SC_63_VHL-MYC-r1",
"20201001_POL_II_Total_DMSO_MP8262_S92": "mp891-MV411_MEF2D_NT_SC_63_DMSO-POLII-r1",
"20201001_POL_II_Total_VHL_MP8262_S98": "mp892-MV411_MEF2D_NT_SC_63_VHL-POLII-r1",}

In [ ]:
for val in a:
    rep = val
    for k,v in rename.items():
        rep = rep.replace(k,v)
    !mv ../../data/$project/fastqs/$val ../../data/$project/fastqs/$rep

In [ ]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [ ]:
gsheet

In [ ]:
a = ! ls ../../data/$project/fastqs
a

In [ ]:
df = {
"fastq_1": [],
"fastq_2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in h.grouped(a,2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('-')[0]]
    df['group'].append("MV4"+row['name'].values[0].split('-r')[0].split('-MV4')[1])
    df['replicate'].append(row['name'].values[0].split('-r')[1])
    df['fastq_1'].append(project+"/fastqs/"+val[0])
    df['fastq_2'].append(project+"/fastqs/"+val[1])
    df['antibody'].append(row['protein'].values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq_1'].append('ref/mp845-MV411-INPUT-r2_R1.fastq.gz')
df['fastq_2'].append('ref/mp845-MV411-INPUT-r2_R2.fastq.gz')
df['antibody'].append("")
df['control'].append("")
df = pd.DataFrame(df)

In [ ]:
df.loc[0].tolist()

In [ ]:
cat ../nextflow/chipseq_MEF2D_degraded_design.csv

In [ ]:
df[df.columns[[3,4,0,1,2,5]]].to_csv('../nextflow/'+project+'_design.csv',index=False)

In [ ]:
! cd ../../data/ && sudo ../nextflow log ## to get access to the previous runs

In [ ]:
#process chips
! cd ../../data/ && sudo ../nextflow run nf-core/chipseq --paired_end --seq_center 'DFCI' --email 'jkobject@gmail.com' --input ../AMLproject/nextflow/$project_design.csv --genome GRCh38 --skip_preseq --max_cpus 16 -profile docker -w work -resume exotic_bartik

In [ ]:
! sudo mv ../../data/results/* ../../data/$project/ && sudo rm -r ..data/work

In [ ]:
ls ../../TrimGalore-0.6.5/trim_galore

In [ ]:
# get scaling values
norm, mapped = h.getSpikeInControlScales(refgenome="../../data/ref/reference_droso.fna",
fastQfolder='../../data/'+project+'/fastqs/',
pairedEnd=True, cores=12,
tofilter=True,
totrim=True,
tomap=True,
results="../../data/"+project+"/",
pathtotrim_galore="../../TrimGalore-0.6.5/trim_galore")
norm, mapped

In [ ]:
mapped

computing scales from the excel sheet

In [ ]:
scales = [1.0,
1.0,
0.9644136372,
0.9068238696,
1.0,
1.0,
0.3954022974,
0.8368988329,
1.0,
1.0,
0.5551449949,
0.844680746]

In [ ]:
bams = ! ls ../../data/$project/bwa/mergedLibrary/*.bam
bams

In [ ]:
bams[6]

### on scalled data

In [ ]:
! mkdir ../../data/$project/diffPeaks/ && ! mkdir ../../data/$project/diffData/

In [ ]:
! mkdir ../../data/$project/droso_aligned

In [ ]:
! mv ../../data/$project/mp* ../../data/$project/droso_aligned

In [ ]:
wigs = ! ls ../../data/$project/bwa/mergedLibrary/bigwig/*.bigWig

In [ ]:
initscales = ! cat ../../data/$project/bwa/mergedLibrary/bigwig/scale/*.txt

In [ ]:
! ls ../../data/$project/bwa/mergedLibrary/bigwig/scale/*.txt

In [ ]:
rescales = [val*float(initscales[1+i]) for i, val in enumerate(scales)]

In [ ]:
rescales

In [ ]:
chip.bigWigFrom(bams[1:], 
                genome='GRCh38',scaling=rescales,
               numthreads=12)

In [ ]:
!mkdir ../../data/$project/recalib_bigwig/ && mv bigwig/* ../../data/$project/recalib_bigwig/

In [ ]:
mv bigwig/* ../../data/$project/recalib_bigwig/

In [ ]:
#! gsutil -m cp -r ../../data/$project/droso_aligned gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/recalib_bigwig gs://amlproject/Chip/$project/

In [ ]:
rm -r ../../data/$project/droso_aligned/

In [ ]:
# diffPeak on scaled data 
for i in range(int((len(bams)-1)/2)):
    if i==4:
        continue
    name1 = bams[i+1]
    name2 = bams[i+7]
    print('\n')
    print(name1,name2)
    print(chip.fullDiffPeak(name1, name2, control1=bams[0], res_directory="../../data/"+project+"/diffPeaks/", scaling=[scales[i],scales[i+6]], directory="../../data/"+project+'/diffData/'))

In [ ]:
os.popen('for i in $(ls ../../data/'+project+'/diffPeaks/*.bed); \
            do echo $(wc -l $i); \
            done').read().split('\n')

In [ ]:
bw = ! ls ../../data/$project/recalib_bigwig/*
bw

In [ ]:
!mkdir ../results/$project/
!mkdir ../results/$project/plots
!mkdir ../results/$project/plots/heatmaps/

In [ ]:
# GENOME WIDE comparison

In [ ]:
peaks = ! ls ../../data/$project/bwa/mergedLibrary/macs/broadPeak/*.broadPeak

In [ ]:
names = ["DMSO_FLAG_MEF2D",
"DMSO_IFR8",
"DMSO_MED1",
"DMSO_MEF2C",
"DMSO_MYC",
"DMSO_POLII",
"VHL_FLAG_MEF2D",
"VHL_IFR8",
"VHL_MED1",
"VHL_MEF2C",
"VHL_MYC",
"VHL_POLII"]

In [ ]:
! mkdir ../../data/$project/peakplot/

In [ ]:
for i, val in enumerate(bw):
    if i <0:
        continue
    name = names[i]
    print(name)
    chip.getPeaksAt(peaks[i], bigwigs = val, bigwignames= name, peaknames=['Macs2_Peaks'], window=3000, folder="", title=name, numthreads=8, refpoint="center", name='../../data/'+project+'/peakplot/'+name+'_mat.pdf', withDeeptools=True, torecompute=True, vmax=2.5, legendLoc="lower-left")

In [ ]:
rm ../../data/chipseq_MEF2D_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL-POLII_R1_tre*

In [ ]:
cond1peak = ! ls ../../data/$project/diffPeaks/*cond1.bed
cond2peak = ! ls ../../data/$project/diffPeaks/*cond2.bed
commonpeak = ! ls ../../data/$project/diffPeaks/*common.bed
cond1peak

In [ ]:
names = ["FLAG_MEF2D",
"IFR8",
"MED1",
"MEF2C",
"MYC",
"POLII"]

In [ ]:
for i in range(int(len(bw)/2)):
    if i <0:
        continue
    cl = len(bw)
    name1 = bw[i]
    name2 = bw[i+6]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
   # for val in peak:
      #  chip.dropWeirdChromosomes(val)
    name = names[i]
    print(name1,name2)
    chip.getPeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, folder="", title=name, numthreads=7, refpoint="center", name='../../data/'+project+'/diffPeaks/'+name+'_mat.pdf', withDeeptools=True, torecompute=True)

In [ ]:
#h.createFoldersFor('../results/'+project+'/plots/scaled/diffPeaks/')
! cp ../../data/$project/diffPeaks/*.pdf ../results/$project/plots/scaled/diffPeaks/

In [ ]:
!cp ../../data/$project/peakplot/*_mat.pdf ../results/$project/plots/scaled/

### looking at TSS

In [11]:
server = BiomartServer( "http://www.ensembl.org/biomart" )
ensembl = server.datasets['hsapiens_gene_ensembl']
ensembltss = pd.read_csv(io.StringIO(ensembl.search({
  'attributes': ['ensembl_gene_id','gene_biotype', "transcription_start_site", "3_utr_start","start_position","external_gene_name", 'chromosome_name']
}, header=1).content.decode()), sep='\t')

/home/jeremie/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
ensmbl.show_attributes_by_page()

In [12]:
MEF2Dtargets = h.fileToList('../results/slamseqMax/MEF2Dtargets.txt')
MEF2Dtargets.pop(0)

'# Filtered by: padj <=0.05 in SLAM-seq at 2 or 24 hours, basemean>10, log2FC SLAM at 2 hours <-0.2'

In [133]:
peaksVHL = pd.read_csv('../../data/chipseq_MEF2D_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-POLII_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL-POLII_R1_treat_pileup_c3.0_cond2.bed', sep='\t',header=None, skiprows=1, names=['chrom','start','end','name','score'])
peaksVHL['chrom']= [i[3:] for i in peaksVHL['chrom']]
peaksVHL[['start','end']] = peaksVHL[['start','end']].astype(int)

In [135]:
peaksDMSO = pd.read_csv('../../data/chipseq_MEF2D_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-POLII_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL-POLII_R1_treat_pileup_c3.0_cond1.bed', sep='\t', header=None, skiprows=1, names=['chrom','start','end','name','score'])
peaksDMSO['chrom']= [i[3:] for i in peaksDMSO['chrom']]
peaksDMSO[['start','end']] = peaksDMSO[['start','end']].astype(int)

In [136]:
peakcommon = pd.read_csv('../../data/chipseq_MEF2D_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-POLII_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL-POLII_R1_treat_pileup_c3.0_common.bed', sep='\t',header=None, skiprows=1, names=['chrom','start','end','name','score'])
peakcommon['chrom']= [i[3:] for i in peakcommon['chrom']]
peakcommon[['start','end']] = peakcommon[['start','end']].astype(int)

In [16]:
ensembltss['Chromosome/scaffold name'] = ensembltss['Chromosome/scaffold name'].astype(str)
ensembltss = ensembltss[ensembltss['Chromosome/scaffold name'].isin(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '3', '4', '5', '6', '7', '8', '9','X','Y'])]
ensembltss = ensembltss.drop_duplicates('Gene start (bp)')
ensembltss = ensembltss.reset_index(drop=True)
ensembltss = ensembltss.astype(str)
ensembltss['loci'] = ensembltss['Transcription start site (TSS)'].astype(int)
ensembltss['chrom'] = ensembltss['Chromosome/scaffold name']
ensembltss = ensembltss.sort_values(by=['chrom','loci']).reset_index(drop=True)

In [143]:
chip.substractPeaksTo(peaksDMSO, ensembltss).to_csv('../../data/'+project+'/diffPeaks/polII_DMSO_TSSonly.bed',sep='\t',index=None,header=False)
chip.substractPeaksTo(peakcommon, ensembltss).to_csv('../../data/'+project+'/diffPeaks/polII_common_TSSonly.bed',sep='\t',index=None,header=False)
chip.substractPeaksTo(peaksVHL, ensembltss).to_csv('../../data/'+project+'/diffPeaks/polII_VHL_TSSonly.bed',sep='\t',index=None,header=False)

In [144]:
chip.getPeaksAt(['../../data/'+project+'/diffPeaks/polII_DMSO_TSSonly.bed', '../../data/'+project+'/diffPeaks/polII_common_TSSonly.bed', '../../data/'+project+'/diffPeaks/polII_VHL_TSSonly.bed'],
                bigwigs = ['../../data/' + project + '/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-POLII_R1.bw', '../../data/' + project + '/recalib_bigwig/MV411_MEF2D_NT_SC_63_VHL-POLII_R1.bw'], 
                bigwignames= ['IRF8_DMSO', 'IRF8_VHL'], 
                peaknames=['DMSO','common','VHL'], window=3000, folder="", title='polII_with_IRF8_degron', numthreads=4, 
                refpoint="center", 
                name='../../data/'+project+'/diffPeaks/polII_with_IRF8_degron_mat.pdf', 
                withDeeptools=True, torecompute=True, legendLoc="lower-left")

CompletedProcess(args='computeMatrix reference-point -S ../../data/chipseq_MEF2D_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-POLII_R1.bw ../../data/chipseq_MEF2D_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_VHL-POLII_R1.bw  --referencePoint center --regionsFileName ../../data/chipseq_MEF2D_degraded/diffPeaks/polII_DMSO_TSSonly.bed ../../data/chipseq_MEF2D_degraded/diffPeaks/polII_common_TSSonly.bed ../../data/chipseq_MEF2D_degraded/diffPeaks/polII_VHL_TSSonly.bed --missingDataAsZero --outFileName ../../data/chipseq_MEF2D_degraded/diffPeaks/polII_with_IRF8_degron_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 4 && plotHeatmap --matrixFile ../../data/chipseq_MEF2D_degraded/diffPeaks/polII_with_IRF8_degron_mat.gz --outFileName ../../data/chipseq_MEF2D_degraded/diffPeaks/polII_with_IRF8_degron_mat.pdf --refPointLabel center --legendLocation lower-left --regionsLabel DMSO common VHL --plotTitle polII_with_IRF8_degron', returncode=0, stdout=b'', stderr=b"One of the group

In [151]:
MEF2Dtss = ensembltss[ensembltss['Gene name'].isin(MEF2Dtargets)].reset_index(drop=True)

In [186]:
a = ! wc -l ../../data/$project/diffPeaks/polII_DMSO_TSSonly.bed 

In [187]:
int(a[0].split(' ')[0])/len(peaksDMSO), len(peaksDMSO)

(0.09885075322255009, 12878)

In [159]:
len(chip.substractPeaksTo(peaksDMSO, MEF2Dtss))/int(a[0].split(' ')[0])

0.04556166535742341

In [184]:
a = ! wc -l ../../data/$project/diffPeaks/polII_common_TSSonly.bed 

In [185]:
int(a[0].split(' ')[0])/ len(peakcommon),  len(peakcommon)

(0.12313008844374158, 85365)

In [161]:
len(chip.substractPeaksTo(peakcommon, MEF2Dtss))/int(a[0].split(' ')[0])

0.006849966701550757

In [181]:
a = ! wc -l ../../data/$project/diffPeaks/polII_VHL_TSSonly.bed 

In [183]:
int(a[0].split(' ')[0])/len(peaksVHL),  len(peaksVHL)

(0.05641025641025641, 390)

In [163]:
len(chip.substractPeaksTo(peaksVHL, MEF2Dtss))/int(a[0].split(' ')[0])

0.0

### on unscalled data

In [ ]:
bams = !ls ../../data/$project/bwa/mergedLibrary/mp*.bam
bams

In [ ]:
! mkdir ../../data/$project/diffPeaks_unscaled

In [ ]:
bams

In [ ]:
#on unscalled data 
for i in range(int(len(bams[1:])/2)):
    if i < 0:
        continue
    name1 = bams[1+i]
    name2 = bams[7+i]
    print(name1,name2)
    chip.fullDiffPeak(name1,name2, control1='../../data/'+project+'/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam', directory = "../../data/"+project+"/diffData_unscaled/", res_directory = "../../data/"+project+"/diffPeaks_unscaled/",pairedend=False)

In [ ]:
bw = ! ls ../../data/$project/bwa/mergedLibrary/bigwig/*.bigWig
bw

In [ ]:
cond1peak = ! ls ../../data/$project/diffPeaks_unscaled/*cond1.bed
cond2peak = ! ls ../../data/$project/diffPeaks_unscaled/*cond2.bed
commonpeak = ! ls ../../data/$project/diffPeaks_unscaled/*common.bed
commonpeak

In [ ]:
names = ["FLAG_MEF2D",
"IFR8",
"MED1",
"MEF2C",
"MYC",
"POLII"]

In [ ]:
for i in range(int(len(bw)/2)):
    name1 = bw[i]
    name2 = bw[6+i]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
    #for val in peak:
     #   chip.dropWeirdChromosomes(val)
    name = names[i]
    print(name1,name2)
    chip.getPeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, folder="", title=name, numthreads=7, torecompute=True, refpoint='center', name='../../data/'+project+'/diffPeaks_unscaled/'+name+'_mat.pdf', withDeeptools=True)

In [ ]:
h.createFoldersFor('../results/'+project+'/plots/unscaled/diffPeaks/')
! cp ../../data/$project/diffPeaks_unscaled/*.pdf ../results/$project/plots/unscaled/diffPeaks/

! gsutil -m cp gs://amlproject/Chip/$project/bwa/mergedLibrary/deepTools/**.pdf ../results/$project/plots/

In [ ]:
! gsutil -m cp -r ../../data/$project/diffPeaks gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/diffData gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/diffPeaks_unscaled gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/diffData_unscaled gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/peakplot gs://amlproject/Chip/$project/

## v7

In [ ]:
project="chipseq_MEF2CMEF2D_degraded"

In [ ]:
! gsutil ls gs://transfer-amlproject/200924_MP8230_fastq/

In [ ]:
mkdir ../data/$project/ && mkdir ../data/$project/qc/

In [ ]:
!gsutil -m cp gs://transfer-amlproject/200924_MP8230_fastq/multiqc_report.html ../data/$project/qc/
!gsutil -m cp -r gs://transfer-amlproject/200924_MP8230_fastq/Reports/ ../data/$project/qc/
!gsutil -m cp -r gs://transfer-amlproject/200924_MP8230_fastq/multiqc_data/ ../data/$project/qc/

In [ ]:
! gsutil -m cp gs://transfer-amlproject/200924_MP8230_fastq/*  gs://transfer-amlproject/$project/

### analysis

In [ ]:
! mkdir ../../data/$project && mkdir ../../data/$project/fastqs && gsutil -m cp gs://transfer-amlproject/$project/* ../../data/$project/fastqs

In [ ]:
! rm ../../data/$project/fastqs/multiqc_report.html

In [ ]:
a = ! ls ../../data/$project/fastqs
a

In [ ]:
rename = {
"20200924_MP1_MP8230_S113": "mp893-MV411_RNP_AAVS1-H3K27AC-r3",
"20200924_MP2_MP8230_S114": "mp894-MV411_RNP_AAVS1-H3K27AC-r4",
"20200924_MP3_MP8230_S115": "mp895-MV411_RNP_MEF2C-H3K27AC-r1",
"20200924_MP4_MP8230_S116": "mp896-MV411_RNP_MEF2C-H3K27AC-r2",
"20200924_MP5_MP8230_S117": "mp897-MV411_RNP_MEF2D-H3K27AC-r3",
"20200924_MP6_MP8230_S118": "mp898-MV411_RNP_MEF2D-H3K27AC-r4",
"20200924_MP7_MP8230_S119": "mp899-MV411_RNP_MEF2C_MEF2D-H3K27AC-r1",
"20200924_MP8_MP8230_S120": "mp900-MV411_RNP_MEF2C_MEF2D-H3K27AC-r2",}

In [ ]:
for val in a:
    rep = val
    for k,v in rename.items():
        rep = rep.replace(k,v)
    !mv ../../data/$project/fastqs/$val ../../data/$project/fastqs/$rep

In [ ]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [ ]:
gsheet

In [ ]:
a = ! ls ../../data/$project/fastqs
a

In [ ]:
df = {
"fastq_1": [],
"fastq_2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in h.grouped(a,2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('-')[0]]
    df['group'].append("MV4"+row['name'].values[0].split('-r')[0].split('-MV4')[1])
    df['replicate'].append(row['name'].values[0].split('-r')[1])
    df['fastq_1'].append(project+"/fastqs/"+val[0])
    df['fastq_2'].append(project+"/fastqs/"+val[1])
    df['antibody'].append(row['protein'].values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq_1'].append('ref/mp845-MV411-INPUT-r2_R1.fastq.gz')
df['fastq_2'].append('ref/mp845-MV411-INPUT-r2_R2.fastq.gz')
df['antibody'].append("")
df['control'].append("")
df = pd.DataFrame(df)

In [ ]:
df

In [ ]:
df[df.columns[[3,4,0,1,2,5]]].to_csv('../nextflow/'+project+'_design.csv',index=False)

In [ ]:
! cd ../../data/ && sudo ../nextflow log ## to get access to the previous runs

In [ ]:
#process chips
! cd ../../data/ && sudo ../nextflow run nf-core/chipseq --paired_end --seq_center 'DFCI' --email 'jkobject@gmail.com' --input ../AMLproject/nextflow/$project_design.csv --genome GRCh38 --skip_preseq --max_cpus 16 -profile docker -w work -resume exotic_bartik

In [ ]:
! gsutil -m cp -r gs://workamlproject/MEF2CMEF2D/output/* gs://amlproject/Chip/chipseq_MEF2CMEF2D_degraded/

In [ ]:
# get scaling values
norm, mapped = h.getSpikeInControlScales(refgenome="../../data/ref/reference_droso.fna",
fastQfolder='../../data/'+project+'/fastqs/',
pairedEnd=True, cores=12,
tofilter=True,
totrim=True,
tomap=True,
results="../../data/"+project+"/",
pathtotrim_galore="../../TrimGalore-0.6.5/trim_galore")
norm, mapped

computing scales from the excel sheet

In [ ]:
scales = [1.00,
1.00,
0.98,
1.08,
0.74,
0.99,
1.13,
0.94]

In [ ]:
#! mkdir ../../data/$project/bwa 
#!mkdir ../../data/$project/bwa/mergedLibrary
#!gsutil cp gs://amlproject/Chip/$project/bwa/mergedLibrary/*.ba* ../../data/$project/bwa/mergedLibrary/
#! gsutil -m cp -r gs://amlproject/Chip/$project/bwa/mergedLibrary/bigwig/ ../../data/$project/bwa/mergedLibrary/
! gsutil  -m cp -r gs://amlproject/Chip/$project/bwa/mergedLibrary/macs/ ../../data/$project/bwa/mergedLibrary/

In [ ]:
bams = ! ls ../../data/$project/bwa/mergedLibrary/*.bam
bams

### on scalled data

In [ ]:
! mkdir ../../data/$project/diffPeaks/
! mkdir ../../data/$project/diffData/
! mkdir ../../data/$project/droso_aligned
! mv ../../data/$project/mp* ../../data/$project/droso_aligned

In [ ]:
wigs = ! ls ../../data/$project/bwa/mergedLibrary/bigwig/*.bigWig
wigs

In [ ]:
# diffPeak on scaled data (full reprocessing)
for i in range(len(bams[3:])):
    if i < 3:
        continue
    bam1 = bams[1+(i%2)]
    bam2 = bams[3+i]
    print(bam1,bam2)
    print(chip.fullDiffPeak(bam1,bam2, control1=bams[0], scaling=[1.0, scales[2+i]], directory = "../../data/"+project+"/diffData/", res_directory = "../../data/"+project+"/diffPeaks/",pairedend=True))

In [ ]:
initscales = ! cat ../../data/$project/bwa/mergedLibrary/bigwig/scale/*.txt
initscales

In [ ]:
rescales = [val*float(initscales[1+i]) for i, val in enumerate(scales)]
rescales

In [ ]:
chip.bigWigFrom(bams[3:], 
                genome='GRCh38',scaling=rescales[2:],
               numthreads=8)

In [ ]:
!mkdir ../../data/$project/recalib_bigwig/ 
!mv bigwig/* ../../data/$project/recalib_bigwig/
!cp ../../data/$project/bwa/mergedLibrary/bigwig/*AAVS1*.bigWig ../../data/$project/recalib_bigwig/

In [ ]:
os.popen('for i in $(ls ../../data/'+project+'/diffPeaks/*.bed); \
            do echo $(wc -l $i); \
            done').read().split('\n')

In [ ]:
bw = ! ls ../../data/$project/recalib_bigwig/*
bw

In [ ]:
! gsutil -m cp -r ../../data/$project/droso_aligned gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/recalib_bigwig gs://amlproject/Chip/$project/

In [ ]:
!mkdir ../results/$project/
!mkdir ../results/$project/plots
!mkdir ../results/$project/plots/heatmaps/

In [ ]:
# GENOME WIDE comparison

In [ ]:
peaks = ! ls ../../data/$project/bwa/mergedLibrary/macs/broadPeak/*.broadPeak

In [ ]:
names = ["AAVS1", "AAVS1_v2", "MEF2C","MEF2C_v2","MEF2C-MEF2D","MEF2C-MEF2D_v2", "MEF2D","MEF2D_v2"]

In [ ]:
for i, val in enumerate(bw):
    if i <0:
        continue
    name = names[i]
    print(name)
    chip.getPeaksAt(peaks[i], bigwigs = val, bigwignames= name, peaknames=['Macs2_Peaks'], window=3000, folder="", title=name, numthreads=8, refpoint="center", name='../../data/'+project+'/peakplot/'+name+'_mat.pdf', withDeeptools=True, torecompute=True, vmax=2.5, legendLoc="lower-left")

In [ ]:
h.createFoldersFor('../results/'+project+'/plots/scaled/heatmaps/')
! cp ../../data/$project/*.pdf ../results/$project/plots/scaled/heatmaps/

In [ ]:
cond1peak = ! ls ../../data/$project/diffPeaks/*cond1.bed
cond2peak = ! ls ../../data/$project/diffPeaks/*cond2.bed
commonpeak = ! ls ../../data/$project/diffPeaks/*common.bed
cond1peak

In [ ]:
for i in range(int(len(bw[2:]))):
    if i <1:
        continue
    name1 = bw[i%2]
    name2 = bw[i+2]
    a = int(i/2)+3 if i%2 else int(i/2)
    peak = [cond1peak[a], commonpeak[a], cond2peak[a]]
   # for val in peak:
      #  chip.dropWeirdChromosomes(val)
    name = names[i+2]
    print(name,name1,name2,a)
    chip.getPeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, folder="", title=name, numthreads=7, refpoint="center", name='../../data/'+project+'/diffPeaks/'+name+'_mat.pdf', withDeeptools=True, torecompute=False)

In [ ]:
! mkdir ../results/additional_degraded_v1/plots/scaled/diffPeaks/
! cp ../../data/additional_degraded_v1/diffPeaks/*.pdf ../results/additional_degraded_v1/plots/scaled/diffPeaks/

### on unscalled data

In [ ]:
bams = !ls ../../data/$project/bwa/mergedLibrary/mp*.bam
bams

In [ ]:
! mkdir ../../data/$project/diffPeaks_unscaled

In [ ]:
#on unscalled data 
for i in range(int(len(bams[3:]))):
    if i < 4:
        continue
    name1 = bams[1+i%2]
    name2 = bams[3+i]
    print(name1,name2)
    chip.fullDiffPeak(name1,name2, control1='../../data/'+project+'/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam', directory = "../../data/"+project+"/diffData_unscaled/", res_directory = "../../data/"+project+"/diffPeaks_unscaled/",pairedend=False)

In [ ]:
bw = ! ls ../../data/$project/bwa/mergedLibrary/bigwig/*.bigWig
bw

In [ ]:
cond1peak = ! ls ../../data/$project/diffPeaks_unscaled/*cond1.bed
cond2peak = ! ls ../../data/$project/diffPeaks_unscaled/*cond2.bed
commonpeak = ! ls ../../data/$project/diffPeaks_unscaled/*common.bed
commonpeak

In [ ]:
for i in range(int(len(bw[2:]))):
    name1 = bw[i%2]
    name2 = bw[2+i]
    a = int(i/2)+3 if i%2 else int(i/2)
    peak = [cond1peak[a], commonpeak[a], cond2peak[a]]
    #for val in peak:
     #   chip.dropWeirdChromosomes(val)
    name = names[i+2]
    print(name1,name2)
    chip.getPeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, folder="", title=name, numthreads=7, torecompute=True, refpoint='center', name='../../data/'+project+'/diffPeaks_unscaled/'+name+'_mat.pdf', withDeeptools=True)

In [ ]:
h.createFoldersFor('../results/'+project+'/plots/unscaled/diffPeaks/')
! cp ../../data/$project/diffPeaks_unscaled/*.pdf ../results/$project/plots/unscaled/diffPeaks/

! gsutil -m cp gs://amlproject/Chip/$project/bwa/mergedLibrary/deepTools/**.pdf ../results/$project/plots/

In [ ]:
! gsutil -m cp -r ../../data/$project/diffPeaks gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/diffData gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/diffPeaks_unscaled gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/diffData_unscaled gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/peakplot gs://amlproject/Chip/$project/

## v8

In [114]:
project="chipseq_IRF8_degraded"

In [ ]:
fastq = ! gsutil ls gs://transfer-amlproject/201023_MP8292_fastq/
fastq

In [ ]:
mkdir ../data/$project/ && mkdir ../data/$project/qc/

In [ ]:
!gsutil -m cp gs://transfer-amlproject/201023_MP8292_fastq/multiqc_report.html ../data/$project/qc/
!gsutil -m cp -r gs://transfer-amlproject/201023_MP8292_fastq/Reports/ ../data/$project/qc/
!gsutil -m cp -r gs://transfer-amlproject/201023_MP8292_fastq/multiqc_data/ ../data/$project/qc/

In [ ]:
! gsutil -m cp gs://transfer-amlproject/201023_MP8292_fastq/*  gs://transfer-amlproject/$project/

### analysis

In [ ]:
rename = {
"20201023_IRF8_DMSO_MP8292_S119": "mp901-MV411_IRF8_NT_DMSO-IRF8-r1",
"20201023_IRF8_VHL_MP8292_S125": "mp902-MV411_IRF8_NT_VHL-IRF8-r1",
"20201023_MED1_DMSO_MP8292_S122": "mp903-MV411_IRF8_NT_DMSO-MED1-r1",
"20201023_MED1_VHL_MP8292_S128": "mp904-MV411_IRF8_NT_VHL-MED1-r1",
"20201023_MEF2C_DMSO_MP8292_S121": "mp905-MV411_IRF8_NT_DMSO-MEF2C-r1",
"20201023_MEF2C_VHL_MP8292_S127": "mp906-MV411_IRF8_NT_VHL-MEF2C-r1",
"20201023_MEF2D_DMSO_MP8292_S120": "mp907-MV411_IRF8_NT_DMSO-MEF2D-r1",
"20201023_MEF2D_VHL_MP8292_S126": "mp908-MV411_IRF8_NT_VHL-MEF2D-r1",
"20201023_MYC_DMSO_MP8292_S123": "mp909-MV411_IRF8_NT_DMSO-MYC-r1",
"20201023_MYC_VHL_MP8292_S129": "mp910-MV411_IRF8_NT_VHL-MYC-r1",
"20201023_POL_II_total_DMSO_MP8292_S124": "mp911-MV411_IRF8_NT_DMSO-POLII_total-r1",
"20201023_POL_II_total_VHL_MP8292_S130": "mp912-MV411_IRF8_NT_VHL-POLII_total-r1",}

In [ ]:
for val in fastq:
    rep = val
    for k,v in rename.items():
        rep = rep.replace(k,v)
    rep = rep.replace('transfer-amlproject/201023_MP8292_fastq/',"amlproject/Chip/"+project+'/fastqs')
    !gsutil cp $val $rep

In [ ]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [ ]:
fastq = ! gsutil ls gs://amlproject/Chip/$project/fastqs/
fastq

In [ ]:
df = {
"fastq_1": [],
"fastq_2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in h.grouped(fastq,2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('-')[0]]
    df['group'].append("MV4"+row['name'].values[0].split('-r')[0].split('-MV4')[1])
    df['replicate'].append(row['name'].values[0].split('-r')[1])
    df['fastq_1'].append(val[0])
    df['fastq_2'].append(val[1])
    df['antibody'].append(row['protein'].values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq_1'].append('gs://amlproject/Chip/IRF2BP2_degraded_rep3/fastqs/mp845-MV411-INPUT-r2_R1.fastq.gz')
df['fastq_2'].append('gs://amlproject/Chip/IRF2BP2_degraded_rep3/fastqs/mp845-MV411-INPUT-r2_R2.fastq.gz')
df['antibody'].append("")
df['control'].append("")
df = pd.DataFrame(df)

In [ ]:
df

In [ ]:
df[df.columns[[3,4,0,1,2,5]]].to_csv('../nextflow/'+project+'_design.csv',index=False)

### about

0. you need to have a google project set up with a billing account
1. you need to activte your APIs this way: https://cloud.google.com/life-sciences/docs/tutorials/nextflow?hl=fr
2. nextflow needs to be installed with this installation command `export NXF_MODE=google && curl https://get.nextflow.io | bash`

In [ ]:
! cd ../../nextflow log ## to get access to the previous runs

In [ ]:
#process chips 
! ../../nextflow run jkobject/chipseq \
    --paired_end \
    --seq_center 'DFCI' \
    --email 'jkobject@gmail.com' \
    --input ../nextflow/chipseq_IRF8_degraded_design.csv \
    --genome GRCh38 \
    --max_cpus 16 \
    -profile jkcloud \ #my profile for the cloud (to edit for your account)
    -w gs://workamlproject/IRF8res \ #where the withinput will be stored
    --spiking \ #I have spiking using drosophilia genome (default)
    --outdir gs://workamlproject/IRF8 \ #specify aa bucket+folderr where the results will be
    --tracedir ../nextflow/IRF8info/ \ #you need to specify a local place for that

In [ ]:
h.createFoldersFor('../../data/'+project+'/bwa/mergedLibrary/')
!gsutil -m cp gs://amlproject/Chip/$project/bwa/mergedLibrary/*.ba* ../../data/$project/bwa/mergedLibrary/
! gsutil -m cp -r gs://amlproject/Chip/$project/bwa/mergedLibrary/bigwig/ ../../data/$project/bwa/mergedLibrary/
! gsutil  -m cp -r gs://amlproject/Chip/$project/bwa/mergedLibrary/macs/ ../../data/$project/bwa/mergedLibrary/

computing scales from the excel sheet

In [ ]:
scales = [
        1.00,
        0.79,
        0.59,
        1.00,
        0.77,
        1.00,
        1.00,
        0.72,
        0.91,
        1.00,
        1.00,
        0.23,
         ]

In [ ]:
bams = ! ls ../../data/$project/bwa/mergedLibrary/*.bam
bams

### on scalled data

In [ ]:
! mkdir ../../data/$project/diffPeaks/ && ! mkdir ../../data/$project/diffData/

In [ ]:
wigs = ! ls ../../data/$project/bwa/mergedLibrary/bigwig/*.bigWig
wigs

In [ ]:
# diffPeak on scaled data (full reprocessing)
for i in range(int(len(bams)/2)):
    if i < 0:
        continue
    bam1 = bams[1+i]
    bam2 = bams[7+i]
    print(bam1,bam2)
    print(chip.fullDiffPeak(bam1,bam2, control1=bams[0], scaling=[scales[(i*2)], scales[(i*2)+1]], directory = "../../data/"+project+"/diffData/", res_directory = "../../data/"+project+"/diffPeaks/",pairedend=True))

In [ ]:
initscales = ! cat ../../data/$project/bwa/mergedLibrary/bigwig/scale/*.txt
initscales

In [ ]:
rescales = [val*float(initscales[1+i]) for i, val in enumerate(scales)]
rescales

In [ ]:
chip.bigWigFrom(bams[1:], 
                genome='GRCh38',scaling=rescales,
               numthreads=8)

In [ ]:
!mkdir ../../data/$project/recalib_bigwig/

In [ ]:
!mv bigwig/* ../../data/$project/recalib_bigwig/

In [ ]:
os.popen('for i in $(ls ../../data/'+project+'/diffPeaks/*.bed); \
            do echo $(wc -l $i); \
            done').read().split('\n')

In [115]:
bw = ! ls ../../data/$project/recalib_bigwig/*
bw

['../../data/chipseq_IRF8_degraded/recalib_bigwig/MV411_IRF8_NT_DMSO-IRF8_R1.bw',
 '../../data/chipseq_IRF8_degraded/recalib_bigwig/MV411_IRF8_NT_DMSO-MED1_R1.bw',
 '../../data/chipseq_IRF8_degraded/recalib_bigwig/MV411_IRF8_NT_DMSO-MEF2C_R1.bw',
 '../../data/chipseq_IRF8_degraded/recalib_bigwig/MV411_IRF8_NT_DMSO-MEF2D_R1.bw',
 '../../data/chipseq_IRF8_degraded/recalib_bigwig/MV411_IRF8_NT_DMSO-MYC_R1.bw',
 '../../data/chipseq_IRF8_degraded/recalib_bigwig/MV411_IRF8_NT_DMSO-POLII_total_R1.bw',
 '../../data/chipseq_IRF8_degraded/recalib_bigwig/MV411_IRF8_NT_VHL-IRF8_R1.bw',
 '../../data/chipseq_IRF8_degraded/recalib_bigwig/MV411_IRF8_NT_VHL-MED1_R1.bw',
 '../../data/chipseq_IRF8_degraded/recalib_bigwig/MV411_IRF8_NT_VHL-MEF2C_R1.bw',
 '../../data/chipseq_IRF8_degraded/recalib_bigwig/MV411_IRF8_NT_VHL-MEF2D_R1.bw',
 '../../data/chipseq_IRF8_degraded/recalib_bigwig/MV411_IRF8_NT_VHL-MYC_R1.bw',
 '../../data/chipseq_IRF8_degraded/recalib_bigwig/MV411_IRF8_NT_VHL-POLII_total_R1.bw']

In [ ]:
!mkdir ../results/$project/
!mkdir ../results/$project/plots
!mkdir ../results/$project/plots/heatmaps/

In [ ]:
# GENOME WIDE comparison

In [ ]:
peaks = ! ls ../../data/$project/bwa/mergedLibrary/macs/broadPeak/*.broadPeak

In [ ]:
names = ["IRF8","MED1","MEF2C","MEF2D","MYC","POLII_total"]

In [ ]:
for i, val in enumerate(bw):
    if i <1:
        continue
    name = names[i-6]+'_wIRF8ko' if i//6 else names[i]+'_DMSO'
    print(name)
    chip.getPeaksAt(peaks[i], bigwigs = val, bigwignames= name, peaknames=['Macs2_Peaks'], window=3000, folder="", title=name, numthreads=8, refpoint="center", name='../../data/'+project+'/'+name+'_mat.pdf', withDeeptools=True, torecompute=True, legendLoc="lower-left")

In [ ]:
h.createFoldersFor('../results/'+project+'/plots/scaled/heatmaps/')
! cp ../../data/$project/*.pdf ../results/$project/plots/scaled/heatmaps/

In [ ]:
cond1peak = ! ls ../../data/$project/diffPeaks/*cond1.bed
cond2peak = ! ls ../../data/$project/diffPeaks/*cond2.bed
commonpeak = ! ls ../../data/$project/diffPeaks/*common.bed
cond1peak

In [ ]:
for i in range(int(len(bw)/2)):
    if i <0:
        continue
    name1 = bw[i]
    name2 = bw[i+6]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
   # for val in peak:
      #  chip.dropWeirdChromosomes(val)
    name = names[i]
    print(name1,name2)
    chip.getPeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, folder="", title=name, numthreads=8, refpoint="center", name='../../data/'+project+'/diffPeaks/'+name+'_mat.pdf', withDeeptools=True, torecompute=False)

In [ ]:
! mkdir ../results/$project/plots/scaled/diffPeaks/
! cp ../../data/$project/diffPeaks/*.pdf ../results/$project/plots/scaled/diffPeaks/

### Looking at TSS

In [12]:
IRF8targets = h.fileToList('../results/slamseqMax/IRF8targets.txt')
IRF8targets.pop(0)

'# Filtered by: padj <=0.05 in SLAM-seq at 2 or 24 hours, basemean>10, log2FC SLAM at 2 hours <-0.2'

In [133]:
peaksVHL = pd.read_csv('../../data/chipseq_MEF2D_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-POLII_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL-POLII_R1_treat_pileup_c3.0_cond2.bed', sep='\t',header=None, skiprows=1, names=['chrom','start','end','name','score'])
peaksVHL['chrom']= [i[3:] for i in peaksVHL['chrom']]
peaksVHL[['start','end']] = peaksVHL[['start','end']].astype(int)

In [135]:
peaksDMSO = pd.read_csv('../../data/chipseq_MEF2D_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-POLII_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL-POLII_R1_treat_pileup_c3.0_cond1.bed', sep='\t', header=None, skiprows=1, names=['chrom','start','end','name','score'])
peaksDMSO['chrom']= [i[3:] for i in peaksDMSO['chrom']]
peaksDMSO[['start','end']] = peaksDMSO[['start','end']].astype(int)

In [136]:
peakcommon = pd.read_csv('../../data/chipseq_MEF2D_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-POLII_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL-POLII_R1_treat_pileup_c3.0_common.bed', sep='\t',header=None, skiprows=1, names=['chrom','start','end','name','score'])
peakcommon['chrom']= [i[3:] for i in peakcommon['chrom']]
peakcommon[['start','end']] = peakcommon[['start','end']].astype(int)

In [16]:
ensembltss['Chromosome/scaffold name'] = ensembltss['Chromosome/scaffold name'].astype(str)
ensembltss = ensembltss[ensembltss['Chromosome/scaffold name'].isin(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '3', '4', '5', '6', '7', '8', '9','X','Y'])]
ensembltss = ensembltss.drop_duplicates('Gene start (bp)')
ensembltss = ensembltss.reset_index(drop=True)
ensembltss = ensembltss.astype(str)
ensembltss['loci'] = ensembltss['Transcription start site (TSS)'].astype(int)
ensembltss['chrom'] = ensembltss['Chromosome/scaffold name']
ensembltss = ensembltss.sort_values(by=['chrom','loci']).reset_index(drop=True)

In [143]:
chip.substractPeaksTo(peaksDMSO, ensembltss).to_csv('../../data/'+project+'/diffPeaks/polII_DMSO_TSSonly.bed',sep='\t',index=None,header=False)
chip.substractPeaksTo(peakcommon, ensembltss).to_csv('../../data/'+project+'/diffPeaks/polII_common_TSSonly.bed',sep='\t',index=None,header=False)
chip.substractPeaksTo(peaksVHL, ensembltss).to_csv('../../data/'+project+'/diffPeaks/polII_VHL_TSSonly.bed',sep='\t',index=None,header=False)

In [144]:
chip.getPeaksAt(['../../data/'+project+'/diffPeaks/polII_DMSO_TSSonly.bed', '../../data/'+project+'/diffPeaks/polII_common_TSSonly.bed', '../../data/'+project+'/diffPeaks/polII_VHL_TSSonly.bed'],
                bigwigs = ['../../data/' + project + '/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-POLII_R1.bw', '../../data/' + project + '/recalib_bigwig/MV411_MEF2D_NT_SC_63_VHL-POLII_R1.bw'], 
                bigwignames= ['IRF8_DMSO', 'IRF8_VHL'], 
                peaknames=['DMSO','common','VHL'], window=3000, folder="", title='polII_with_IRF8_degron', numthreads=4, 
                refpoint="center", 
                name='../../data/'+project+'/diffPeaks/polII_with_IRF8_degron_mat.pdf', 
                withDeeptools=True, torecompute=True, legendLoc="lower-left")

CompletedProcess(args='computeMatrix reference-point -S ../../data/chipseq_MEF2D_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-POLII_R1.bw ../../data/chipseq_MEF2D_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_VHL-POLII_R1.bw  --referencePoint center --regionsFileName ../../data/chipseq_MEF2D_degraded/diffPeaks/polII_DMSO_TSSonly.bed ../../data/chipseq_MEF2D_degraded/diffPeaks/polII_common_TSSonly.bed ../../data/chipseq_MEF2D_degraded/diffPeaks/polII_VHL_TSSonly.bed --missingDataAsZero --outFileName ../../data/chipseq_MEF2D_degraded/diffPeaks/polII_with_IRF8_degron_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 4 && plotHeatmap --matrixFile ../../data/chipseq_MEF2D_degraded/diffPeaks/polII_with_IRF8_degron_mat.gz --outFileName ../../data/chipseq_MEF2D_degraded/diffPeaks/polII_with_IRF8_degron_mat.pdf --refPointLabel center --legendLocation lower-left --regionsLabel DMSO common VHL --plotTitle polII_with_IRF8_degron', returncode=0, stdout=b'', stderr=b"One of the group

In [151]:
MEF2Dtss = ensembltss[ensembltss['Gene name'].isin(MEF2Dtargets)].reset_index(drop=True)

In [186]:
a = ! wc -l ../../data/$project/diffPeaks/polII_DMSO_TSSonly.bed 

In [187]:
int(a[0].split(' ')[0])/len(peaksDMSO), len(peaksDMSO)

(0.09885075322255009, 12878)

In [159]:
len(chip.substractPeaksTo(peaksDMSO, MEF2Dtss))/int(a[0].split(' ')[0])

0.04556166535742341

In [184]:
a = ! wc -l ../../data/$project/diffPeaks/polII_common_TSSonly.bed 

In [185]:
int(a[0].split(' ')[0])/ len(peakcommon),  len(peakcommon)

(0.12313008844374158, 85365)

In [161]:
len(chip.substractPeaksTo(peakcommon, MEF2Dtss))/int(a[0].split(' ')[0])

0.006849966701550757

In [181]:
a = ! wc -l ../../data/$project/diffPeaks/polII_VHL_TSSonly.bed 

In [183]:
int(a[0].split(' ')[0])/len(peaksVHL),  len(peaksVHL)

(0.05641025641025641, 390)

In [163]:
len(chip.substractPeaksTo(peaksVHL, MEF2Dtss))/int(a[0].split(' ')[0])

0.0

### on unscalled data

In [ ]:
! mkdir ../../data/$project/diffPeaks_unscaled

In [ ]:
#on unscalled data 
for i in range(int(len(bams)/2)):
    if i < 0:
        continue
    name1 = bams[i]
    name2 = bams[6+i]
    print(name1,name2)
    chip.fullDiffPeak(name1,name2, control1='../../data/'+project+'/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam', directory = "../../data/"+project+"/diffData_unscaled/", res_directory = "../../data/"+project+"/diffPeaks_unscaled/",pairedend=False)

In [ ]:
bw = ! ls ../../data/$project/bwa/mergedLibrary/bigwig/*.bigWig
bw

In [ ]:
cond1peak = ! ls ../../data/$project/diffPeaks_unscaled/*cond1.bed
cond2peak = ! ls ../../data/$project/diffPeaks_unscaled/*cond2.bed
commonpeak = ! ls ../../data/$project/diffPeaks_unscaled/*common.bed
commonpeak

In [ ]:
for i in range(int((len(bw)-1)/2)):
    name1 = bw[i]
    name2 = bw[6+i]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
    #for val in peak:
     #   chip.dropWeirdChromosomes(val)
    name = names[i]
    print(name1,name2)
    chip.getPeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, folder="", title=name, numthreads=8, torecompute=True, refpoint='center', name='../../data/'+project+'/diffPeaks_unscaled/'+name+'_mat.pdf', withDeeptools=True)

In [ ]:
h.createFoldersFor('../results/'+project+'/plots/unscaled/diffPeaks/')
! cp ../../data/$project/diffPeaks_unscaled/*.pdf ../results/$project/plots/unscaled/diffPeaks/

! gsutil -m cp gs://amlproject/Chip/$project/bwa/mergedLibrary/deepTools/**.pdf ../results/$project/plots/

In [ ]:
! gsutil -m cp -r ../../data/$project/diffPeaks gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/diffData gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/diffPeaks_unscaled gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/diffData_unscaled gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/peakplot gs://amlproject/Chip/$project/